In [1]:
using Zygote
using Statistics: var, mean
using LinearAlgebra: norm
using Flux.NNlib: relu
using Flux.Optimise

┌ Info: Recompiling stale cache file /home/leo/.julia/compiled/v1.1/Zygote/4kbLI.ji for Zygote [e88e6eb3-aa80-5325-afca-941959d7151f]
└ @ Base loading.jl:1184
┌ Info: Recompiling stale cache file /home/leo/.julia/compiled/v1.1/Flux/QdkVy.ji for Flux [587475ba-b771-5e3f-ad9e-33799f191a9c]
└ @ Base loading.jl:1184


In [2]:
function myvar(v)
    mv  = mean(v)
    sum((v .- mv).^2)./(length(v)-1)
end

myvar (generic function with 1 method)

In [3]:
function train(params)
    opt = ADAM(0.01)
    V = 8
    maxiter = 20000
    msk = [false, true, true, true, false, true, true, true]
    pp = params[:,msk]
    for i=1:maxiter
        grad = view(loss'(params), :,msk)
        Optimise.update!(opt, pp, grad)
        view(params, :, msk) .= pp
        if i%100 == 0
            @show loss(params)
        end
    end
    params
end

train (generic function with 1 method)

In [4]:
L1 = [(1,6), (2,7), (3,8), (4,9), (5,10), (1,2), (2,3), (3,4), (4,5), (5,1), (6,8), (8,10), (10,7), (7,9), (9,6)]
L1 = [i<j ? (i,j) : (j,i) for (i,j) in L1]

15-element Array{Tuple{Int64,Int64},1}:
 (1, 6) 
 (2, 7) 
 (3, 8) 
 (4, 9) 
 (5, 10)
 (1, 2) 
 (2, 3) 
 (3, 4) 
 (4, 5) 
 (1, 5) 
 (6, 8) 
 (8, 10)
 (7, 10)
 (7, 9) 
 (6, 9) 

In [5]:
LL = Any[]
for i=1:9
    for j=i+1:10
        push!(LL, (i,j))
    end
end

In [6]:
L2 = setdiff(LL, L1)

30-element Array{Any,1}:
 (1, 3) 
 (1, 4) 
 (1, 7) 
 (1, 8) 
 (1, 9) 
 (1, 10)
 (2, 4) 
 (2, 5) 
 (2, 6) 
 (2, 8) 
 (2, 9) 
 (2, 10)
 (3, 5) 
 ⋮      
 (4, 7) 
 (4, 8) 
 (4, 10)
 (5, 6) 
 (5, 7) 
 (5, 8) 
 (5, 9) 
 (6, 7) 
 (6, 10)
 (7, 8) 
 (8, 9) 
 (9, 10)

In [7]:
function loss(x)
    a = [norm(x[:,i]-x[:,j]) for (i, j) in L1]
    b = [norm(x[:,i]-x[:,j]) for (i, j) in L2]
    myvar(a) + myvar(b) + exp(relu(-mean(b) + mean(a) + 0.1))
end

loss (generic function with 1 method)

In [8]:
function train(params)
    opt = ADAM(0.001)
    maxiter = 20000
    msk = fill(true, size(params, 2))
    msk[[1, 2]] .= false
    pp = params[:,msk]
    for i=1:maxiter
        grad = view(loss'(params), :,msk)
        Optimise.update!(opt, pp, grad)
        view(params, :, msk) .= pp
        if i%100 == 0
            @show loss2(params)
        end
    end
    params
end

train (generic function with 1 method)

In [9]:
params = randn(5, 10)
params = train(params)

UndefVarError: UndefVarError: loss2 not defined

In [10]:
[norm(params[:,i]-params[:,j]) for (i,j) in L1]

15-element Array{Float64,1}:
 2.5352978678369626
 3.759871275463382 
 3.140700087731001 
 2.7665935887717046
 2.855794900937913 
 2.246092457428752 
 2.0673530706598924
 2.589412691869277 
 3.964400157115386 
 2.024468153870198 
 4.730835119709707 
 2.712549617973731 
 3.24318812003815  
 2.998984520516086 
 2.517110456016402 

In [11]:
[norm(params[:,i]-params[:,j]) for (i,j) in L2]

30-element Array{Float64,1}:
 1.6540618898534392
 2.338751108120379 
 2.9242589346718235
 3.6070036483573764
 1.2406923703551158
 1.7030769977253208
 3.767908659114869 
 2.2215203674642767
 2.1164308139312125
 4.541465291371022 
 1.9214169362375377
 2.9961162384583364
 2.467703437175878 
 ⋮                 
 2.6046519183357937
 3.2437829941193015
 2.5105898369618647
 3.051799581953192 
 4.060141395569454 
 4.255606667990122 
 2.7638939328504937
 3.4616686898001316
 3.6369845242044785
 2.635014220963385 
 4.170334999895718 
 2.2759021638383445